<a href="https://colab.research.google.com/github/hru003/Learning/blob/main/Learning/Apache_beam/Simple_ETL_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install apache-beam with pip.
!pip install --quiet apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 33.8 MB/s eta 0:00:00


In [2]:
import apache_beam as beam
import sqlite3
from apache_beam.options.pipeline_options import PipelineOptions

In [22]:
!wget https://raw.githubusercontent.com/hru003/Learning/master/Apache_beam/data.csv

--2023-08-11 11:33:20--  https://raw.githubusercontent.com/hru003/Learning/master/Apache_beam/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301 [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]     301  --.-KB/s    in 0s      

2023-08-11 11:33:20 (8.37 MB/s) - ‘data.csv’ saved [301/301]



In [31]:
with beam.Pipeline() as pipeline:
  (pipeline
    | "Read data" >> beam.io.ReadFromText("data.csv")
    | "Extract Lines" >> beam.Map(lambda x: x.split(','))
    | "Convert Age to int" >> beam.Map(lambda x: [x[0],int(x[1])])
    | beam.Map(print)
  )
  pipeline.run()

['John', 22]
['Mary', 28]
['Peter', 33]
['Jane', 37]
['David', 41]
['Sally', 45]
['Michael', 23]
['Susan', 29]
['Richard', 35]
['Elizabeth', 39]
['Thomas', 43]
['Jennifer', 47]
['Charles', 24]
['Sarah', 30]
['William', 36]
['Christine', 40]
['Christopher', 44]
['Nancy', 25]
['Daniel', 31]
['Emily', 37]
['George', 41]
['Ethan', 45]
['Jessica', 26]
['Alexander', 32]
['Megan', 38]
['Mark', 42]
['Nicole', 46]
['John', 22]
['Mary', 28]
['Peter', 33]
['Jane', 37]
['David', 41]
['Sally', 45]
['Michael', 23]
['Susan', 29]
['Richard', 35]
['Elizabeth', 39]
['Thomas', 43]
['Jennifer', 47]
['Charles', 24]
['Sarah', 30]
['William', 36]
['Christine', 40]
['Christopher', 44]
['Nancy', 25]
['Daniel', 31]
['Emily', 37]
['George', 41]
['Ethan', 45]
['Jessica', 26]
['Alexander', 32]
['Megan', 38]
['Mark', 42]
['Nicole', 46]


In [33]:
with beam.Pipeline() as pipeline:
  (pipeline
    | "Read data" >> beam.io.ReadFromText("data.csv")
    | "Extract Lines" >> beam.Map(lambda x: x.split(','))
    | "Convert Age to int" >> beam.Map(lambda x: [x[0],int(x[1])])
    | "Filter records with age less than 35" >> beam.Filter(lambda x:x[1] < 35)
    | beam.Map(print)
  )
  pipeline.run()

['John', 22]
['Mary', 28]
['Peter', 33]
['Michael', 23]
['Susan', 29]
['Charles', 24]
['Sarah', 30]
['Nancy', 25]
['Daniel', 31]
['Jessica', 26]
['Alexander', 32]
['John', 22]
['Mary', 28]
['Peter', 33]
['Michael', 23]
['Susan', 29]
['Charles', 24]
['Sarah', 30]
['Nancy', 25]
['Daniel', 31]
['Jessica', 26]
['Alexander', 32]


In [13]:
conn = sqlite3.connect("my_database.sqlite3")
c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS {} (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)".format('AGE_Lesser_35'))
c.execute("CREATE TABLE IF NOT EXISTS {} (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)".format('AGE_Greater_35'))
conn.commit()

In [51]:
def write_data(data):
  c.execute("INSERT INTO {} (name, age) VALUES (?, ?)".format('AGE_Lesser_35'), data)
  conn.commit()

with beam.Pipeline() as pipeline:
  (pipeline
    | "Read data" >> beam.io.ReadFromText("data.csv")
    | "Extract Lines" >> beam.Map(lambda x: x.split(','))
    | "Convert Age to int" >> beam.Map(lambda x: [x[0],int(x[1])])
    | "Filter records with age less than 35" >> beam.Filter(lambda x:x[1] < 35)
    | "Write data" >> beam.Map(write_data)
  )
  pipeline.run()

In [52]:
c.execute("SELECT * FROM {}".format('AGE_Lesser_35'))
rows = c.fetchall()
for row in rows:
  print(row)

(1, 'John', 22)
(2, 'Mary', 28)
(3, 'Peter', 33)
(4, 'Michael', 23)
(5, 'Susan', 29)
(6, 'Charles', 24)
(7, 'Sarah', 30)
(8, 'Nancy', 25)
(9, 'Daniel', 31)
(10, 'Jessica', 26)
(11, 'Alexander', 32)
(12, 'John', 22)
(13, 'Mary', 28)
(14, 'Peter', 33)
(15, 'Michael', 23)
(16, 'Susan', 29)
(17, 'Charles', 24)
(18, 'Sarah', 30)
(19, 'Nancy', 25)
(20, 'Daniel', 31)
(21, 'Jessica', 26)
(22, 'Alexander', 32)


In [53]:
def write_data(data):
  c.execute("INSERT INTO {} (name, age) VALUES (?, ?)".format('AGE_Greater_35'), data)
  conn.commit()

with beam.Pipeline() as pipeline:
  (pipeline
    | "Read data" >> beam.io.ReadFromText("data.csv")
    | "Extract Lines" >> beam.Map(lambda x: x.split(','))
    | "Convert Age to int" >> beam.Map(lambda x: [x[0],int(x[1])])
    | "Filter records with age less than 35" >> beam.Filter(lambda x:x[1] > 35)
    | "Write data" >> beam.Map(write_data)
  )
  pipeline.run()

In [54]:
c.execute("SELECT * FROM {}".format('AGE_Greater_35'))
rows = c.fetchall()
for row in rows:
  print(row)

(1, 'Jane', 37)
(2, 'David', 41)
(3, 'Sally', 45)
(4, 'Elizabeth', 39)
(5, 'Thomas', 43)
(6, 'Jennifer', 47)
(7, 'William', 36)
(8, 'Christine', 40)
(9, 'Christopher', 44)
(10, 'Emily', 37)
(11, 'George', 41)
(12, 'Ethan', 45)
(13, 'Megan', 38)
(14, 'Mark', 42)
(15, 'Nicole', 46)
(16, 'Jane', 37)
(17, 'David', 41)
(18, 'Sally', 45)
(19, 'Elizabeth', 39)
(20, 'Thomas', 43)
(21, 'Jennifer', 47)
(22, 'William', 36)
(23, 'Christine', 40)
(24, 'Christopher', 44)
(25, 'Emily', 37)
(26, 'George', 41)
(27, 'Ethan', 45)
(28, 'Megan', 38)
(29, 'Mark', 42)
(30, 'Nicole', 46)
